In [1]:
import sympy as sp
import numpy as np
from sympy import *
from IPython.display import Math, Latex

## List of tuning parameters for loiter mode
1. $ANG\_MAX$ : Loiter angle max [deg] $(AP \; value: 0)$ <br>
&emsp; <b>AP definition:</b> Loiter maximum lean angle. Set to zero for 2/3 of PSC_ANGLE_MAX or ANGLE_MAX. <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the maximum angle that can be commanded by the pilot. It is currently limited to 45 degrees.</i> </font> <br>
</br>
2. $LOITER\_SPEED\_DEFAULT$ : Loiter horizontal maximum speed [cm/s] $(AP \; value: 800)$ <br>
&emsp; <b>AP definition:</b> Defines the maximum speed in cm/s which the aircraft will travel horizontally while in loiter mode. <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the maximum horizontal speed that the UAV can travel at while in loiter mode.</i> </font> <br>
</br>
3. $LOITER\_SPEED\_MIN$ : Loiter horizontal minimum speed [cm/s] $(AP \; value: -/20)$ <br>
&emsp; <b>AP definition:</b> Defines the minimum speed in cm/s which the aircraft will travel horizontally while in loiter mode. <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the minimum horizontal speed that the UAV can travel at while in loiter mode.</i> </font> <br>
</br>
4. $LOITER\_ACCEL\_MAX\_DEFAULT$ : Loiter maximum correction acceleration [cm/s/s] $(AP \; value: 150)$ <br>
&emsp; <b>AP definition:</b> Loiter maximum correction acceleration in cm/s/s.  Higher values cause the copter to correct position errors more aggressively. <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the maximum acceleration error that the controller will correct.</i> </font> <br>
</br>
5. $LOITER\_BRAKE\_ACCEL\_DEFAULT$ : Loiter braking acceleration [cm/s/s] $(AP \; value: 150)$ <br>
&emsp; <b>AP definition:</b> Loiter braking acceleration in cm/s/s. Higher values stop the copter more quickly when the stick is centered. <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the maximum brake acceleration that the controller will apply on the UAV to stop it at the target stopping location.</i> </font> <br>
</br>
6. $LOITER\_BRAKE\_JERK\_DEFAULT$ : Loiter braking jerk [cm/s/s/s] $(AP \; value: 500)$ <br>
&emsp; <b>AP definition:</b> Loiter braking jerk in cm/s/s/s. Higher values will remove braking faster if the pilot moves the sticks during a braking maneuver. <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the maximum acceleration change that can be allowed in a controller time step so as to prevent abrupt jerks during the UAV motion.</i> </font> <br>
</br>
7. $LOITER\_BRAKE\_START\_DELAY\_DEFAULT$ : Loiter brake start delay [s] $(AP \; value: 0)$ <br>
&emsp; <b>AP definition:</b> Loiter brake start delay. <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the time delay between the centering of the stick and the actual start of the brake application.</i> </font> <br>
</br>
8. $LOITER\_VEL\_CORRECTION\_MAX$ : Maximum speed used to correct position errors in loiter [cm/s] $(AP \; value: -/200)$ <br>
&emsp; <b>AP definition:</b> - <br>
<font color = red> <i><b>Interpretation:</b> This parameter defines the maximum velocity error that the controller will correct.</i> </font> <br>

## Calculating the maximum allowable position error (using the inverse square root controller)

$inv\_sqrt\_controller$ = Inverse square root controller <br>
$curr\_vel$ = Current velocity

<b>Stopping distance</b> = $inv\_sqrt\_controller$ ($curr\_vel$, $LOITER\_ACCEL\_MAX\_DEFAULT$, $K_{p,xy}$) <br>
</br>
0 $\leq$ $curr\_vel$ $\leq$ $LOITER\_VEL\_CORRECTION\_MAX$ <br>
</br>
$linear\_velocity$ = $\frac{LOITER\_ACCEL\_MAX\_DEFAULT}{K_{p,xy}}$ <br>
</br>
if ($curr\_vel$ < $linear\_velocity$) <br>
{<br>
&emsp; <b>Stopping distance</b> = $\frac{curr\_vel}{K_{p,xy}}$ <br>
}<br>
else <br>
{<br>
&emsp; $linear\_dist$ = $\frac{LOITER\_ACCEL\_MAX\_DEFAULT}{K_{p,xy}^2}$ <br>
</br>
&emsp; <b>Stopping distance</b> = $\frac{linear\_dist}{2} + \frac{(curr\_vel)^2}{2\cdot LOITER\_ACCEL\_MAX\_DEFAULT}$ <br>
}

## Calculating the new target position, velocity, and acceleration after the application of brake (centering of stick)

$gnd\_speed\_limit\_cms$ = max (min ($LOITER\_SPEED\_DEFAULT$, $ekfGndSpdLimit \cdot 100$), $LOITER\_SPEED\_MIN$) <br>
$pilot\_acceleration\_max$ = $g \cdot 100 \cdot tan(rad(ANG\_MAX))$ <br>

$desired\_vel$ = target velocity calculated from the position controller <br>
$predicted\_accel$ = target acceleration calculated from the position controller <br>
$desired\_accel$ = desired acceleration at the instant of brake application (centering of stick) <br>

$desired\_vel$ += $predicted\_accel \cdot dt$ <br>
$desired\_speed = desired\_vel.length()$ <br>

if ($desired\_speed$ != $0$) <br>
{ <br>
&emsp; $desired\_vel\_norm = \frac{desired\_vel}{desired\_speed}$ <br>

&emsp; $drag\_decel = \frac{pilot\_acceleration\_max \cdot desired\_speed}{gnd\_speed\_limit\_cms}$ <br>

&emsp; $loiter\_brake\_accel = 0$ <br>

&emsp; if ($desired\_accel = 0$) <br>
&emsp; { <br>
&emsp; &emsp; if ($(curr\_time - brake\_timer) > LOITER\_BRAKE\_START\_DELAY\_DEFAULT \cdot 1000$) <br>
&emsp; &emsp; { <br>
&emsp; &emsp; &emsp; &emsp; $brake\_gain = K_{p,\dot{x}\dot{y}} \cdot 0.5$ <br>
&emsp; &emsp; &emsp; &emsp; $loiter\_brake\_accel$ = constrain ($sqrt\_controller (desired\_speed, brake\_gain, LOITER\_BRAKE\_JERK\_DEFAULT, dt), 0, LOITER\_BRAKE\_ACCEL\_DEFAULT$) <br>
&emsp; &emsp; } <br>
&emsp; } <br>
&emsp; else <br>
&emsp; { <br>
&emsp; &emsp; &emsp; $loiter\_brake\_accel = 0$ <br>
&emsp; &emsp; &emsp; $brake\_timer = curr\_time$ <br>
&emsp; } <br>
&emsp; $brake\_accel$ += constrain ($loiter\_brake\_accel - brake\_accel, -(LOITER\_BRAKE\_JERK\_DEFAULT \cdot dt), (LOITER\_BRAKE\_JERK\_DEFAULT \cdot dt)$) <br>
&emsp; $loiter\_accel\_brake = desired\_vel\_norm \cdot brake\_accel$ <br>

&emsp; $desired\_speed$ = max ($desired\_speed - (drag\_decel + brake\_accel) \cdot dt, 0$) <br>
&emsp; $desired\_vel = desired\_vel\_norm \cdot desired\_speed$ <br>
} <br>

$desired\_accel$ -= $loiter\_accel\_brake$ <br>

$target\_pos$ += $desired\_vel \cdot dt$ <br>

The new target position, velocity, and acceleration are $target\_pos$, $desired\_vel$, and $desired\_accel$ respectively.

## Calculating the brake acceleration (using the square root controller)

$loiter\_brake\_accel$ = constrain ($sqrt\_controller(desired\_speed, brake\_gain, LOITER\_BRAKE\_JERK\_DEFAULT, dt), 0, LOITER\_BRAKE\_ACCEL\_DEFAULT$) <br>

$linear\_dist = \frac{LOITER\_BRAKE\_JERK\_DEFAULT}{(brake\_gain)^2}$ <br>

if ($desired\_speed > linear\_dist$) <br>
{ <br>
&emsp; $correction\_rate = safe\_sqrt (2 \cdot LOITER\_BRAKE\_JERK\_DEFAULT \cdot (desired\_speed - \frac{linear\_dist}{2}))$ <br>
} <br>
else <br>
if ($desired\_speed < -linear\_dist$) <br>
{ <br>
&emsp; $correction\_rate = -safe\_sqrt (2 \cdot LOITER\_BRAKE\_JERK\_DEFAULT \cdot (-desired\_speed - \frac{linear\_dist}{2}))$ <br>
} <br>
else <br>
{ <br>
&emsp; $correction\_rate = desired\_speed \cdot brake\_gain$ <br>
} <br>

return constrain ($correction\_rate, \frac{-abs(desired\_speed)}{dt}, \frac{abs(desired\_speed)}{dt}$)